<a href="https://colab.research.google.com/github/mingliangwsu/datasharing/blob/master/WWS_spatial_datasets_liu_use_geemap_04011023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#USE gee to show the location and relative data resources for WWS project

##Import the gee API

In [1]:
import ee

##Authenticate and initialize

In [2]:
ee.Authenticate()
#Initialize the library
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Q5LV9UYqD4ooywrW4S3VbnC1_gQO7KMn6Ue-oFUQVAY&tc=8EHjq-M50I_eGkBR791VKGje8uwH5x-3MW8iOr1zxNY&cc=dvBV-QQMxXiyjVHrjlx_N0wPlLmHHvk2YQAccp1X_qo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk67B-UDUKi2GlCEmPuqt-N1vZ4lfXc1dgZbV8xc3H1jOfee_0fGtB0

Successfully saved authorization token.


##Test the API

In [3]:
# Print the elevation of Mount Everest.
dem30m = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem30m.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


##Set boundary for WWS project

In [8]:
#Pacific Nowthwest
pnwxmin,pnwymin,pnwxmax,pnwymax = -126.0,41.5,-110.5,49.0
#for display purposes
wwsbnd = ee.Geometry.Rectangle([pnwxmin,pnwymin,pnwxmax,pnwymax], 'EPSG:4326')
wwscenter = ee.Geometry.Point((pnwxmin+pnwxmax)/2.0,(pnwymin+pnwymax)/2.0)
# dem 
dem10m = ee.Image('USGS/3DEP/10m').clip(wwsbnd)
# State boundary
statebnd = ee.FeatureCollection("TIGER/2016/States").filterBounds(wwsbnd)
# county boundary 
countybnd = ee.FeatureCollection(ee.FeatureCollection("TIGER/2016/Counties")).filterBounds(wwsbnd)
# MTBS burnt area
MTBS = ee.FeatureCollection("USFS/GTAC/MTBS/burned_area_boundaries/v1").filterBounds(wwsbnd)
# NLCD 2019
NLCD2019 = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD").filter(ee.Filter.eq('system:index', '2019')).first().select('landcover').clip(wwsbnd)
# Global Surface Water Mapping Layers
surfacew = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").clip(wwsbnd)
# HUC 8
HUC08 = ee.FeatureCollection("USGS/WBD/2017/HUC08").filterBounds(wwsbnd)
# HUC 10
HUC10 = ee.FeatureCollection("USGS/WBD/2017/HUC10").filterBounds(wwsbnd)
# HUC 12
HUC12 = ee.FeatureCollection("USGS/WBD/2017/HUC12").filterBounds(wwsbnd)

#https://www.oregon.gov/deq/wq/programs/pages/dwp-maps.aspx
#Surface water drinking water source areas in Oregon
OR_SW_DWSAs = ee.FeatureCollection("projects/ee-mingliangliuearth/assets/OR_SW_DWSAs_ORLAMBERT_Ver10_15JAN2019")
#WA
WA_SW_DWSAs = ee.FeatureCollection("projects/ee-mingliangliuearth/assets/WA_drinking_Water_source_area")


#EPA reach ER1-2
StreamEr1_2 = ee.FeatureCollection("projects/ee-mingliangliuearth/assets/Erf1_20") #.filterBounds(wwsbnd)

#WA WQ stations from WADOE
WA_WQ = ee.FeatureCollection("projects/ee-mingliangliuearth/assets/WA_WQ_points_WADOE_pro")
#OR WQ stations from ORDEQ
OR_WQ = ee.FeatureCollection("projects/ee-mingliangliuearth/assets/OR_WQ_points_ORDEQ_pro")


if True:
  gagesII = ee.FeatureCollection("projects/ee-mingliangliuearth/assets/gagesII_9322_sept30_2011").filterBounds(wwsbnd)
  # Load the USGS stream gauges observation data
  #my_map.add_ee_layer(gagesII, {'color': 'blue','width': 3}, 'Gages-II')
  ref_gages = gagesII.filter(ee.Filter.eq('CLASS', 'Ref'))
  noref_gages_gt100km = gagesII.filter(ee.Filter.neq('CLASS', 'Ref')).filter(ee.Filter.gt('DRAIN_SQKM', 250))
  noref_gages_lt100km = gagesII.filter(ee.Filter.neq('CLASS', 'Ref')).filter(ee.Filter.lt('DRAIN_SQKM', 250))
  #count = ref_gages.size().getInfo()
  print('Number of Gages Ref:', ref_gages.size().getInfo(), 
        ' NoRef (>250km2):',noref_gages_gt100km.size().getInfo(),
        ' NoRef (<250km2):',noref_gages_lt100km.size().getInfo())

Number of Gages Ref: 303  NoRef (>250km2): 507  NoRef (<250km2): 186


##Interactive map

In [ ]:
#!pip install folium
!pip install folium --upgrade
try:
  import rasterio
except ImportError as e:  
  !pip install rasterio
  import rasterio

!pip show folium
from pickle import FALSE
# Import the Folium library.
import folium
from folium import plugins
from folium.plugins import MeasureControl
from folium.plugins import BeautifyIcon
import json
import pandas as pd

try:
  import geemap
except ImportError as e:
  !pip install geemap
  import geemap
!pip show geemap

##Mount google drive

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Some basic visual paramaters & functions

In [38]:
#'fillColor': last 2 digit control the fill color opacity, e.g. "00" is 100% transparent, "80": 50% transparent 

poly_params_mode = {
    'color': '000000', 
    'pointSize': 3,
    'pointShape': 'circle',
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000'
}

black_poly_params = {
    'color': '000000', 
    'pointSize': 3,
    'pointShape': 'circle',
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}
white_poly_params = {
    'color': 'FFFFFF', 
    'pointSize': 5,
    'pointShape': 'circle',
    'width': 5,
    'lineType': 'dotted',
    'fillColor': '00000000',
}
red_poly_params = {
    'color': 'FF0000', 
    'pointSize': 3,
    'pointShape': 'circle',
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}
gray_poly_params = {
    'color': '808080', 
    'pointSize': 2,
    'pointShape': 'circle',
    'width': 1,
    'lineType': 'solid',
    'fillColor': '00000000',
}

dem_vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['#0000ff', '#0040ff', '#0080ff', '#00bfff', '#00ffbf', '#00ff80', '#80ff00', '#bfff00'
    , '#ffff00', '#ffbf00', '#ff8000', '#ff4000', '#ff0000', '#bf0040', '#800080', '#400080', '#0000ff', '#4d4dff', '#9999ff', '#e6e6ff']
}

def simplify(feature):
    return feature.setGeometry(feature.geometry().simplify(maxError=30))

def simplify100(feature):
    return feature.setGeometry(feature.geometry().simplify(maxError=100))

###Interactive map

In [39]:
Map = geemap.Map(center=((pnwymin+pnwymax)/2.0, (pnwxmin+pnwxmax)/2.0), zoom=6)
Map.add_basemap('HYBRID')
Map.add_basemap('TERRAIN')
Map.add_basemap('NLCD 2019 CONUS Land Cover')
Map.addLayer(statebnd.style(**white_poly_params), {}, 'US States')

Map.addLayer(HUC12.style(**gray_poly_params), {}, 'HUC12')
Map.addLayer(HUC10.style(**black_poly_params), {}, 'HUC10')






#display(Map)

##Add MTBS data

In [40]:

import datetime
#add new time field
def add_time_ymd(feat):
    time_ymd = ee.Date(feat.get('Ig_Date')).format('YYYY-MM-dd')
    newtime=feat.set("Ig_TimeYMD",time_ymd)
    return newtime

MTBS = MTBS.map(add_time_ymd)
#simplify
#maxError is in meters

MTBS = MTBS.map(simplify100)
 
#start_date = ee.Date('2010-01-01')
start_date = ee.Date('2010-01-01')
end_date = ee.Date('2020-12-31')
#mill second after 2010
start_unix_epoch_timestamp = int(start_date.getInfo()['value'])
end_unix_epoch_timestamp = int(end_date.getInfo()['value'])

fire_size_ac_gt = 5000
selected_MTBS = MTBS.filter(ee.Filter.eq('Incid_Type', 'Wildfire')) \
                    .filter(ee.Filter.gt('BurnBndAc', fire_size_ac_gt)) \
                    .filter(ee.Filter.gt('Ig_Date',start_unix_epoch_timestamp)) \
                    .filter(ee.Filter.lt('Ig_Date', end_unix_epoch_timestamp))
print('Number of all MTBS polygons:', selected_MTBS.size().getInfo())

colorparam = red_poly_params.copy()
colorparam['fillColor'] = 'FF000080'


Map.addLayer(selected_MTBS.style(**colorparam), {}, 'MTBS_2010_2020')

Number of all MTBS polygons: 581


##Add Oregon and Washington drinking water source areas

In [41]:
#add Oregon surface & ground drinking water source areas
OR_SW_DWSAs = OR_SW_DWSAs.map(simplify)
#add WA surface & ground drinking water source areas
WA_SW_DWSAs = WA_SW_DWSAs.map(simplify100)

blue = poly_params_mode.copy()
blue['color'] = '00FFFF'
blue['fillColor'] = 'BFEFFF80'

Map.addLayer(OR_SW_DWSAs.style(**blue), {}, 'Surface water source OR')
Map.addLayer(WA_SW_DWSAs.style(**blue), {}, 'Surface water source WA')



##Add USGS Gages map

In [44]:
Map.addLayer(ref_gages, {'color': 'blue','pointSize': 10,'width': 15}, "GAGESII ref")
Map.addLayer(noref_gages_gt100km, {'color': 'black','pointSize': 10,'width': 5}, "GAGESII nonref>250km2")
Map.addLayer(noref_gages_lt100km, {'color': 'gray','pointSize': 10,'width': 1}, "GAGESII nonref<250km2")

display(Map)

Map(bottom=96529.0, center=[43.096470518529905, -118.07144165039064], controls=(WidgetControl(options=['positi…

##Save map as html

Warning: Seems not entirely functional, i.e. sometimes some maps cannot show up.

In [43]:
html_file = "/content/drive/MyDrive/WWS/index2.html"
Map.to_html(filename=html_file, title='WWS Datasets', width='100%', height='880px')